<a href="https://colab.research.google.com/github/margaritamayoral/Angular-myemail/blob/master/Prueba_DS_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer caso
Supón que tienes una población que se etiqueta en 3 casos a, b, y c. Para dicha población cuentas con 500 variables que pueden afectar su comportamiento y que pueden cambiar a los que son a y pueden pasar a b o c, o cualquiera dependiendo de los cambios de las variables. Describe que ML aplicarías y por que? Qué pruebas estadísticas usarías para comprender que el modelo es consistente. Define 3 métricas para ponderar el error.


In [ ]:
### Preparacion de los datos
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
### Entrenamiento del modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

### Division del conjunto de datos en datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing, considerando que tenemos variables categoricas y numericas.
# Esta funcion realiza el preproceso de las variables para poder despues
# utilizarla en un pipeline junto con el entrenamiento del modelo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definicion de la funcion modelo a utilizar durante el entrenamiento.
# Consiste en una  pipeline que inclye el preprocesamiento de los datos y
# la llamada al tipo de modelo a entrenar.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Entrenamiento del modelo
model.fit(X_train, y_train)

## Evaluacion del modelo

# Realizando las inferencias y evaluando
y_pred = model.predict(X_test)

# Metricas
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
kappa = cohen_kappa_score(y_test, y_pred)

print(f'Accuracy: {accuracy}, F1 Score: {f1}, Cohen\'s Kappa: {kappa}')

# Cross-validation:
cross_val_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f'Cross-validation scores: {cross_val_scores}')


# Segundo caso

En Wal-Mart hay diferentes fuentes de las cuales se ha obtenido un dataset con 120 campos, en el cual se incluye la variable independiente, sin embargo de los 119 restantes hay 69 variables categóricas y el resto son alfanuméricas. Explica si descartarías variables o no y el por que; supongamos que de las 69 variables categóricas 60 son significativas, como las transformarías para que algún modelo pueda ser entrenado con ellas. Los dos puntos anteriores se cumplen si la variable objetivo es continua o discreta?


In [ ]:
# Manipulacion de los datos
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Para entrenar el modelo
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score

# Llamando df al dataset, donde 'target' es la variable objetivo
X = df.drop(columns=['target'])
y = df['target']

# Como primer paso, identificamos todas las variables categoricas y alfanumericas
# existentes dentro del dataset

categorical_features = [...]  # Lista de las 69 variables categoricas
alphanumeric_features = [...]  # Lista de las variables categoricas restantes

# Funciones para preprocesar los datos y utilizar en la pipeline principal
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
# Asumiendo que las variables alfanumericas pueden ser codificadas directamente
alphanumeric_transformer = LabelEncoder()

# Haciendo el preprocesamiento usnado ColumnTransformer para construir una
# pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        # se incluiria una transformacion para las variables alfanumericas si es
        # necesaria
    ])

# Transformamos los datos
X_preprocessed = preprocessor.fit_transform(X)


# Entrenando el modelo

# Separamos el dataset en datos para training y test
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Determinamos si la variable objetivo es continua o discreta y definimos el
# modelo a entrenar en base a la observacion de la variable objetivo
if y.dtype == 'float':
    # Modelo de regresion
    model = RandomForestRegressor(random_state=42)
else:
    # Modelo de clasificacion
    model = RandomForestClassifier(random_state=42)

# Entrenamos el modelo
model.fit(X_train, y_train)

# Realizamos la inferencia y evaluamos
y_pred = model.predict(X_test)

# En el caso de un modelo de regresion
if y.dtype == 'float':
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'MSE: {mse}, R2: {r2}')

# En el caso de un modelo de clasificacion
else:
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}, F1 Score: {f1}')

# Relaizamos la Cross-validation
cross_val_scores = cross_val_score(model, X_preprocessed, y, cv=5, scoring='accuracy' if y.dtype != 'float' else 'neg_mean_squared_error')
print(f'Cross-validation scores: {cross_val_scores}')



# Problema de ML inciso e.

Si tuvieras una base de datos de más de 1000 millones de datos, pero no tienes ninguna
herramienta de big data para procesarla, ¿Qué harías para analizar la información,
procesar datos, generar un modelo y probarlo en un ambiente productivo?

Dada la restricción que se enfrenta con una base de datos de más de mil millones de registros y sin acceso a herramientas de big data, se puede seguir un enfoque estratégico para analizar, procesar y modelar los datos de manera eficiente.
1. Muestreo de Datos
Dado que procesar todo el conjunto de datos en una sola máquina no es práctico, se puede usar técnicas de muestreo para crear un subconjunto representativo de los datos:
Muestreo Aleatorio: Extraer una muestra aleatoria del conjunto de datos. Se necesita  que la muestra sea lo suficientemente grande como para capturar los patrones subyacentes.
Muestreo Estratificado: Si los datos tienen diferentes estratos (por ejemplo, categorías, clases), hay que asegurarse de que la muestra mantenga las proporciones de cada estrato.
2. Preprocesamiento de Datos
Una vez obtenida una muestra manejable, se puede preprocesar los datos usando herramientas como Pandas:


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
import joblib
from flask import Flask, request, jsonify

# Cargar una muestra del conjunto de datos
sample_data = pd.read_csv('sample_data.csv')

# Limpieza y preprocesamiento de datos
sample_data = sample_data.dropna()  # Manejo de valores faltantes
sample_data = pd.get_dummies(sample_data)  # Codificación one-hot de variables categóricas


3. Ingeniería de Características
Crear nuevas características o transformar las existentes para mejorar el rendimiento del modelo:

In [ ]:
# Ejemplo de creación de nuevas características
sample_data['new_feature'] = sample_data['existing_feature1'] / sample_data['existing_feature2']


4. Entrenamiento del Modelo
Seleccionar un modelo de aprendizaje automático adecuado y entrenarlo con los datos muestreados. Usar algoritmos eficientes que puedan manejar grandes conjuntos de datos cuando sea necesario:

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = sample_data.drop('target', axis=1)
y = sample_data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un Clasificador Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Precisión: {accuracy}, Puntaje F1: {f1}')


5. Validación del Modelo
Usar validación cruzada para asegurar que el modelo se generalice bien:

In [ ]:
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f'Puntajes de validación cruzada: {cv_scores}')

6. Escalamiento
Para manejar el conjunto de datos completo de forma incremental o en bloques:

Procesamiento por Lotes: Procesar los datos en lotes. Entrenar el modelo de manera incremental o usar técnicas como el aprendizaje en línea.
Procesamiento Distribuido: Utilizar marcos como Dask o Apache Spark (si están disponibles) para paralelizar el procesamiento de datos y el entrenamiento del modelo.
7. Pruebas y Despliegue
Después de desarrollar el modelo, desplegarlo en un entorno de producción usando herramientas que faciliten el servicio del modelo:

In [ ]:
# Guardar el modelo
joblib.dump(model, 'model.pkl')

# Crear una API simple con Flask para las predicciones del modelo
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    prediction = model.predict(pd.DataFrame(data))
    return jsonify(prediction.tolist())

if __name__ == '__main__':
    app.run(port=5000, debug=True)


8. Monitoreo y Reentrenamiento
Configurar el monitoreo para rastrear el rendimiento del modelo y reentrenarlo según sea necesario:

Monitoreo: Utilizar herramientas como Prometheus y Grafana para monitorear el rendimiento de la API y la precisión del modelo.
Reentrenamiento: Reentrenar periódicamente el modelo utilizando nuevas muestras de datos para asegurar que siga siendo preciso y relevante.

# 3. PySpark

Escribe las sentencias en PySpark para seleccionar toda la información en la tabla “tabla_1”, si dicha tabla tuviera un campo llamado “campo_cat”. Escribe las sentencia para seleccionar todos los valores distintos y para contar la frecuencia de cada uno de estos. Escribe la sentencia para seleccionar los primeros 100 registros, y los últimos 100 registros. Escribe la sentencia para contar todos los registros de la tabla.



Seleccionando toda la información de "Tabla_1"


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number


# Crear una sesión de Spark
spark = SparkSession.builder.appName("SeleccionarInformacion").getOrCreate()

# Cargar la tabla "Tabla_1"
tabla_1 = spark.table("Tabla_1")

# Seleccionar toda la información de la tabla
tabla_1.show()


Contar la frecuencia de cada valor en "campo_cat"


In [ ]:
if "campo_cat" in tabla_1.columns:
    # Contar la frecuencia de cada valor en "campo_cat"
    frecuencias = tabla_1.groupBy("campo_cat").count()
    frecuencias.show()


Seleccionar los primeros 100 registros


In [ ]:
# Seleccionar los primeros 100 registros
primeros_100 = tabla_1.limit(100)
primeros_100.show()


Seleccionar los últimos 100 registros
Para seleccionar los últimos 100 registros, primero necesitamos conocer el número total de registros y luego restar 100 para obtener los últimos registros.

In [ ]:
# Contar el número total de registros
total_registros = tabla_1.count()

# Crear una columna con el número de fila
windowSpec = Window.orderBy("some_column")  # Reemplaza "some_column" con una columna adecuada para el orden
tabla_1_con_fila = tabla_1.withColumn("row_num", row_number().over(windowSpec))

# Seleccionar los últimos 100 registros
ultimos_100 = tabla_1_con_fila.filter(tabla_1_con_fila.row_num > (total_registros - 100))
ultimos_100.show()


Contar todos los registros en la tabla


In [ ]:
# Contar todos los registros en la tabla
conteo_total = tabla_1.count()
print(f"Total de registros en la tabla: {conteo_total}")


# PySpark inciso b

Crear las tablas en PySpark
Primero, vamos a crear las tablas Customer_info y Customer_purchs en PySpark:

In [ ]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CustomerQueries").getOrCreate()

# Datos para Customer_info
data_customer_info = [
    ("1sd", "A", 1),
    ("2sd", "B", 1),
    ("3sd", "A", 0),
    ("45sd", "C", 0),
    ("9sd", "C", 1)
]

# Datos para Customer_purchs
data_customer_purchs = [
    ("1sd", "21-09-19", "Aaabbb", 3, 5.5),
    ("1sd", "21-09-19", "bbbccc", 1, 80),
    ("1sd", "22-09-19", "Cccaaa", 4, 17.5),
    ("2sd", "23-09-19", "Dddaaa", 12, 7.5),
    ("2sd", "23-09-19", "Aaabbb", 2, 5.5),
    ("3sd", "21-09-19", "Aaabbb", 6, 5.5),
    ("3sd", "21-09-19", "bbbccc", 6, 80)
]

# Crear DataFrames
columns_customer_info = ["Id_cliente", "City_chr", "Gndr_chr"]
columns_customer_purchs = ["Id_cliente", "date_chr", "Prod_chr", "Qty_num", "Price_num"]

df_customer_info = spark.createDataFrame(data_customer_info, columns_customer_info)
df_customer_purchs = spark.createDataFrame(data_customer_purchs, columns_customer_purchs)

# Crear tablas temporales
df_customer_info.createOrReplaceTempView("Customer_info")
df_customer_purchs.createOrReplaceTempView("Customer_purchs")


## Género que compra más artículos, la ciudad que compró más, y el cliente que compró más


In [ ]:
query_c = """
WITH total_purchases AS (
    SELECT
        cp.Id_cliente,
        ci.Gndr_chr,
        ci.City_chr,
        SUM(cp.Qty_num) AS total_qty
    FROM Customer_purchs cp
    JOIN Customer_info ci ON cp.Id_cliente = ci.Id_cliente
    GROUP BY cp.Id_cliente, ci.Gndr_chr, ci.City_chr
),
gender_purchase AS (
    SELECT Gndr_chr, SUM(total_qty) AS total_gender_qty
    FROM total_purchases
    GROUP BY Gndr_chr
),
city_purchase AS (
    SELECT City_chr, SUM(total_qty) AS total_city_qty
    FROM total_purchases
    GROUP BY City_chr
),
customer_purchase AS (
    SELECT Id_cliente, SUM(total_qty) AS total_customer_qty
    FROM total_purchases
    GROUP BY Id_cliente
)

SELECT 'Género' AS Category, Gndr_chr AS Identifier, total_gender_qty AS Total
FROM gender_purchase
UNION ALL
SELECT 'Ciudad' AS Category, City_chr AS Identifier, total_city_qty AS Total
FROM city_purchase
UNION ALL
SELECT 'Cliente' AS Category, Id_cliente AS Identifier, total_customer_qty AS Total
FROM customer_purchase
ORDER BY Category, Total DESC
"""

result_c = spark.sql(query_c)
result_c.show()


## Promedio de compra por ciudad


In [ ]:
query_d = """
SELECT
    ci.City_chr,
    AVG(cp.Qty_num * cp.Price_num) AS avg_purchase_per_city
FROM Customer_purchs cp
JOIN Customer_info ci ON cp.Id_cliente = ci.Id_cliente
GROUP BY ci.City_chr
"""

result_d = spark.sql(query_d)
result_d.show()


## Tabla ordenada por fecha, de compra total por cliente y ciudad


In [ ]:
query_e = """
SELECT
    ci.Id_cliente,
    ci.City_chr,
    cp.date_chr,
    SUM(cp.Qty_num * cp.Price_num) AS total_purchase
FROM Customer_purchs cp
JOIN Customer_info ci ON cp.Id_cliente = ci.Id_cliente
GROUP BY ci.Id_cliente, ci.City_chr, cp.date_chr
ORDER BY cp.date_chr, ci.Id_cliente, ci.City_chr
"""

result_e = spark.sql(query_e)
result_e.show()
